## Imports

In [314]:
import os
import wave
import numpy as np

## Deleteing unneeded folders

In [315]:
mainFolderLocation = "ABI-1 Corpus/accents/"
listOfAccents = os.listdir(mainFolderLocation)

# Goes through list of accent folders
for accent in listOfAccents:
    accentFolderLocation = mainFolderLocation + accent + "/"
    if accent == ".DS_Store":
        os.remove(accentFolderLocation)

    accentContent = os.listdir(accentFolderLocation)

    # Goes through list of gender folders
    for genders in accentContent:
        genderFolderLocation = accentFolderLocation + genders + "/"
        if genders == ".DS_Store":
          os.remove(genderFolderLocation)

        genderContent = os.listdir(genderFolderLocation)
        
        # Goes through list of participant folders
        for participant in genderContent:
            participantFolderLocation = genderFolderLocation + participant + "/"
            if participant == ".DS_Store":
                os.remove(participantFolderLocation)

            participantContent = os.listdir(participantFolderLocation)

            for file in participantContent:
                fileAddress = participantFolderLocation + file
                if file[-3:] != "wav":
                    os.remove(fileAddress)

                if file[:12] != "shortpassage":
                    os.remove(fileAddress)


## Obtaining List of directories

In [316]:
def getSpeakerRoots(datapath = "ABI-1 Corpus\\accents"):
    speakerList = []
    accentSubfolder = [f.path for f in os.scandir(datapath) if f.is_dir()]
    for accent in accentSubfolder:
        for gender in ["female", "male"]:
            speakerFolders = os.listdir(os.path.join(accent, gender))
            for speaker in speakerFolders:
                if not speaker.startswith("."):
                    speakerList.append(os.path.join(accent, gender, speaker))
        
    return speakerList

In [317]:
speakerList = getSpeakerRoots()
if "ABI-1 Corpus\\accents\\roi_001\\female\\axm001Copy" in speakerList:
    print("Hel")

Hel


## Getting .wav files

In [318]:
def getWavFile(datapath):
    files = os.listdir(datapath)
    filesWav = [i for i in files if i.endswith(".wav")]
    return filesWav

In [319]:
speakers = []
for i in speakerList:
    speakers.append(i[i.rfind("\\")+1:])

speakerFilePairs = {}
for i in range(len(speakers)):
    speakerFilePairs[speakers[i]] = [getWavFile(speakerList[i]), speakerList[i]]

## Segmenting the data

In [320]:
folderName = "SegmentedDataset"
if os.path.exists(folderName) == False:
    os.mkdir(folderName)

for speaker in list(speakerFilePairs.keys()):
    speakerFolder = folderName + "\\" + speaker
    if os.path.exists(speakerFolder) == False:
        os.mkdir(speakerFolder)

In [321]:
def splitWavFile(inputFile, seconds):
    for recording in inputFile[0]:
        fileLocation = inputFile[1] + "\\" + recording
        with wave.open(fileLocation, "rb") as wavFile:
            params = wavFile.getparams()
            sampleRate = params.framerate
            totalFrames = params.nframes

            # Calculate the number of frames per segment
            framesPerSegment = seconds * sampleRate
            totalSegment = totalFrames // framesPerSegment
            
            for segmentIndex in range(totalSegment):
                startFrame = segmentIndex * framesPerSegment
                wavFile.setpos(startFrame)
                audioData = np.frombuffer(wavFile.readframes(framesPerSegment), dtype=np.int16)
                fileName = inputFile[1][inputFile[1].rfind("\\")+1:]
                outputFile = "SegmentedDataset\\" + fileName + "\\chunk" + str(len(os.listdir("SegmentedDataset\\" + fileName))) + ".wav"

                with wave.open(outputFile, "wb") as outputWav:
                    outputWav.setparams(params)
                    outputWav.writeframes(audioData.tobytes())

            startFrame = totalSegment*framesPerSegment
            wavFile.setpos(startFrame)
            audioData = np.frombuffer(wavFile.readframes(framesPerSegment), dtype=np.int16)
            outputFile = "SegmentedDataset\\" + fileName + "\\chunk" + str(len(os.listdir("SegmentedDataset\\" + fileName))) + ".wav"
            
            with wave.open(outputFile, "wb") as outputWav:
                outputWav.setparams(params)
                outputWav.writeframes(audioData.tobytes())

In [322]:
# rebootFiles = True
rebootFiles = False
if len(os.listdir("SegmentedDataset")[-1]) == 0 or rebootFiles == True:
    indexes = list(speakerFilePairs.keys())

    for index in indexes:
        folderAddress = "SegmentedDataset\\" + index + "\\"
        for file in os.listdir(folderAddress):
            os.remove(folderAddress + file)
    
    for index in indexes:
        splitWavFile(speakerFilePairs[index], 3)

## Getting each data point

In [326]:
dataLocations = []
for speaker in os.listdir(folderName):
    for file in os.listdir(folderName + "\\" + speaker):
        dataLocations.append((speaker, folderName + "\\" + speaker + "\\" + file))

# print(len(dataLocations))

In [327]:
fileThatDontMatch = []
rootFolder = "ABI-1 Corpus\\accents\\"
for accent in os.listdir(rootFolder):
    accentFolder = rootFolder + accent + "\\"
    for gender in os.listdir(accentFolder):
        genderFolder = accentFolder + gender + "\\"
        for speaker in os.listdir(genderFolder):

            address = genderFolder + speaker + "\\"
            if speaker == "axm001-1":
                # Actual
                listOfFiles = os.listdir(address)
                actualTotal = 0
                for i in listOfFiles:
                    with wave.open(address + i, 'rb') as wav_file:
                        num_frames = wav_file.getnframes()
                        frame_rate = wav_file.getframerate()
                        duration = num_frames / float(frame_rate)
                        actualTotal += duration
                
                print(actualTotal)

                # Chunks
                address = "SegmentedDataset\\" + speaker + "\\"
                listOfFiles = os.listdir(address)
                chunksTotal = 0
                for i in listOfFiles:
                    with wave.open(address + i, 'rb') as wav_file:
                        num_frames = wav_file.getnframes()
                        frame_rate = wav_file.getframerate()
                        duration = num_frames / float(frame_rate)
                        chunksTotal += duration

                print(chunksTotal)
                
                if actualTotal != chunksTotal:
                    fileThatDontMatch.append([speaker, actualTotal - chunksTotal])

print("Number of Files that dont match: ", len(fileThatDontMatch))

Number of Files that dont match:  0


## Creating Dataset Object